In [68]:
import paho.mqtt.client as mqtt
import json
import numpy as np
import xgboost as xgb
from numpy import linalg
import socket
from scipy.stats import zscore

HOST = '127.0.0.1'    # The remote host
PORT = 50007              # The same port as used by the server

In [69]:
class KalmanFilter(object):
    def __init__(self, F = None, B = None, H = None, Q = None, R = None, P = None, x0 = None):

        if(F is None or H is None):
            raise ValueError("Set proper system dynamics.")

        self.n = F.shape[1]
        self.m = H.shape[1]

        self.F = F
        self.H = H
        self.B = 0 if B is None else B
        self.Q = np.eye(self.n) if Q is None else Q
        self.R = np.eye(self.n) if R is None else R
        self.P = np.eye(self.n) if P is None else P
        self.x = np.zeros((self.n, 1)) if x0 is None else x0

    def predict(self, u = 0):
        self.x = np.dot(self.F, self.x) + np.dot(self.B, u)
        self.P = np.dot(np.dot(self.F, self.P), self.F.T) + self.Q
        return self.x

    def update(self, z):
        y = z - np.dot(self.H, self.x)
        S = self.R + np.dot(self.H, np.dot(self.P, self.H.T))
        K = np.dot(np.dot(self.P, self.H.T), np.linalg.inv(S))
        self.x = self.x + np.dot(K, y)
        I = np.eye(self.n)
        self.P = np.dot(np.dot(I - np.dot(K, self.H), self.P), 
        	(I - np.dot(K, self.H)).T) + np.dot(np.dot(K, self.R), K.T)



In [70]:
def pm(angle, deg = False):
    ''' Mantém o ângulo entre +- 180/pi '''
    factor = 180 if deg else np.pi

    if angle > factor:
        while angle > factor:
            angle -= 2*factor
    if angle < factor:
        while angle < - factor:
            angle += 2*factor
    return angle

pm = np.vectorize(pm)

# Regressor

In [65]:
model = xgb.XGBRegressor()
model.load_model("../ML/xgboost/xbg_regressor.json")

dt = 1.0/100
F = np.array([[1, dt, 0], [0, 1, dt], [0, 0, 1]])
H = np.array([1, 0, 0]).reshape(1, 3)
Q = np.array([[0.05, 0.05, 0.0], [0.05, 0.05, 0.0], [0.0, 0.0, 0.0]])
R = np.array([0.5]).reshape(1, 1)

kf = KalmanFilter(F = F, H = H, Q = Q, R = R)

# Callback chamado quando o cliente recebe a resposta CONNACK do servidor.
def on_connect(client, userdata, flags, rc):
    print("Conectado com o código de resultado "+str(rc))
    client.subscribe("/gw/ac233fc09283/status")
    
    
# Callback chamado quando uma mensagem PUBLISH é recebida do servidor.
#antenna array configuration: 2,2,0,5,6,1,12,14

def on_message(client, userdata, msg):
    
    global pds
    
    data = json.loads(msg.payload)


    phases = np.array([np.angle(complex(data[1]["aoa"]["iq"][i],data[1]["aoa"]["iq"][i+1])) for i in range(0,len(data[1]["aoa"]["iq"]),2)  ])

    # #Without kalman
    # a2a1 = pm(phases[9::3] - phases[8:-1:3])
    # a3a2 = pm(phases[10::3] - phases[9::3])
    # a3a1 = pm(phases[10::3] - phases[8:-1:3])

    # model_predictions= []

    # for i in range(len(a2a1)) :
    #     model_predictions.append(model.predict(np.array([a2a1[i],a3a2[i],a3a1[i]]).reshape(1,3)))
    # model_predictions=np.array(model_predictions)
    # #print(f"zscore={zscore(model_predictions)}")
    # print(model_predictions)
    # print(f"Media {np.mean(model_predictions)} \t Media filtrada {np.mean(model_predictions[abs(zscore(model_predictions)) <2])}")

    a2a1 = (pm(phases[9::3] - phases[8:-1:3]))
    a3a2 = (pm(phases[10::3] - phases[9::3]))
    a3a1 = (pm(phases[10::3] - phases[8:-1:3]))

    pds = np.dot(H,kf.predict((0,0,0)))[0]
    kf.update((np.mean(a2a1),np.mean(a3a2),np.mean(a3a1)))     

    angle_kalman = model.predict(pds.reshape(1,3))
    
    model_predictions= []
    for i in range(len(a2a1)) :
        model_predictions.append(model.predict(np.array([a2a1[i],a3a2[i],a3a1[i]]).reshape(1,3)))
    
    # with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    #     i=0
    #     while i<100000:
    #         try:
    #             s.connect((HOST, PORT))
    #             s.sendall(str(angle).encode())
    #             data = s.recv(1024)
    #             break
    #         except:
    #             i+=1

    print(f"Model ={np.mean(model_predictions)} \t Kalman={angle_kalman} \t Both = {np.mean([np.mean(model_predictions),angle_kalman])}")


client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

# client.connect("broker.hivemq.com", 1883, 60)
client.connect("test.mosquitto.org", 1883, 60)

client.loop_forever()

Conectado com o código de resultado 0
Model =76.17176055908203 	 Kalman=[40.855545] 	 Both = [58.513653]
Model =87.20010375976562 	 Kalman=[93.439] 	 Both = [90.31955]
Model =91.37555694580078 	 Kalman=[173.55092] 	 Both = [132.46324]
Model =97.67626190185547 	 Kalman=[158.64911] 	 Both = [128.16269]
Model =77.8236312866211 	 Kalman=[90.06369] 	 Both = [83.943665]
Model =76.95398712158203 	 Kalman=[153.43335] 	 Both = [115.193665]
Model =89.86243438720703 	 Kalman=[64.067474] 	 Both = [76.96495]
Model =89.26512145996094 	 Kalman=[155.97208] 	 Both = [122.6186]
Model =91.403564453125 	 Kalman=[161.09018] 	 Both = [126.24687]
Model =77.66059112548828 	 Kalman=[86.52004] 	 Both = [82.09032]
Model =88.10942840576172 	 Kalman=[160.75116] 	 Both = [124.4303]
Model =88.58731079101562 	 Kalman=[177.20348] 	 Both = [132.89539]
Model =91.9483871459961 	 Kalman=[152.1532] 	 Both = [122.0508]
Model =75.74806213378906 	 Kalman=[87.88959] 	 Both = [81.818825]
Model =86.63677215576172 	 Kalman=[153.0

KeyboardInterrupt: 

# Classifier

In [ ]:
model = xgb.XGBRegressor()
model.load_model("../ML/xgboost/xbg_regressor.json")

dt = 1.0/100
F = np.array([[1, dt, 0], [0, 1, dt], [0, 0, 1]])
H = np.array([1, 0, 0]).reshape(1, 3)
Q = np.array([[0.05, 0.05, 0.0], [0.05, 0.05, 0.0], [0.0, 0.0, 0.0]])
R = np.array([0.5]).reshape(1, 1)

kf = KalmanFilter(F = F, H = H, Q = Q, R = R)

# Callback chamado quando o cliente recebe a resposta CONNACK do servidor.
def on_connect(client, userdata, flags, rc):
    print("Conectado com o código de resultado "+str(rc))
    client.subscribe("/gw/ac233fc09283/status")
    
    
# Callback chamado quando uma mensagem PUBLISH é recebida do servidor.
#antenna array configuration: 2,2,0,5,6,1,12,14

def on_message(client, userdata, msg):
    
    global pds
    
    data = json.loads(msg.payload)


    phases = np.array([np.angle(complex(data[1]["aoa"]["iq"][i],data[1]["aoa"]["iq"][i+1])) for i in range(0,len(data[1]["aoa"]["iq"]),2)  ])

    # #Without kalman
    # a2a1 = pm(phases[9::3] - phases[8:-1:3])
    # a3a2 = pm(phases[10::3] - phases[9::3])
    # a3a1 = pm(phases[10::3] - phases[8:-1:3])

    # model_predictions= []

    # for i in range(len(a2a1)) :
    #     model_predictions.append(model.predict(np.array([a2a1[i],a3a2[i],a3a1[i]]).reshape(1,3)))
    # model_predictions=np.array(model_predictions)
    # #print(f"zscore={zscore(model_predictions)}")
    # print(model_predictions)
    # print(f"Media {np.mean(model_predictions)} \t Media filtrada {np.mean(model_predictions[abs(zscore(model_predictions)) <2])}")

    a2a1 = (pm(phases[9::3] - phases[8:-1:3]))
    a3a2 = (pm(phases[10::3] - phases[9::3]))
    a3a1 = (pm(phases[10::3] - phases[8:-1:3]))

    pds = np.dot(H,kf.predict((0,0,0)))[0]
    kf.update((np.mean(a2a1),np.mean(a3a2),np.mean(a3a1)))     

    angle_kalman = model.predict(pds.reshape(1,3))
    
    model_predictions= []
    for i in range(len(a2a1)) :
        model_predictions.append(model.predict(np.array([a2a1[i],a3a2[i],a3a1[i]]).reshape(1,3)))
    
    # with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    #     i=0
    #     while i<100000:
    #         try:
    #             s.connect((HOST, PORT))
    #             s.sendall(str(angle).encode())
    #             data = s.recv(1024)
    #             break
    #         except:
    #             i+=1

    print(f"Model ={np.mean(model_predictions)} \t Kalman={angle_kalman} \t Both = {np.mean([np.mean(model_predictions),angle_kalman])}")


client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

# client.connect("broker.hivemq.com", 1883, 60)
client.connect("test.mosquitto.org", 1883, 60)

client.loop_forever()

array([3.4043527], dtype=float32)

array([[0., 0., 0.]])